In [1]:
import tensorflow as tf
import cv2 as cv
import numpy as np

lines = r'C:\Users\maksym.kashchuk\Desktop\auto-os-ml\datasets\CuLane\test\driver_37_30frame\05181520_0219.MP4\00000.lines.txt'
image = r'C:\Users\maksym.kashchuk\Desktop\auto-os-ml\datasets\CuLane\test\driver_37_30frame\05181520_0219.MP4\00000.jpg'

def data_process_mask(lines_file, image_file):
    mask_wdith, mask_height, _ = cv.imread(image_file).shape

    lines = []

    mask = np.zeros((mask_wdith, mask_height), dtype=np.uint8)

    line_id = lane_id = coords_first_id = coords_second_id = 0

    with open(lines_file, 'r') as file:
        lines = [list(map(int, map(float, line.split()))) for line in file]
    
    while line_id != len(lines) - 1:
        coords_first_id = coords_second_id = 0

        while coords_first_id + 4 <= len(lines[line_id]) or coords_second_id + 4 <= len(lines[line_id + 1]):
            x1, y1 = lines[line_id][coords_first_id], lines[line_id][coords_first_id + 1]
            x2, y2 = lines[line_id][coords_first_id + 2], lines[line_id][coords_first_id + 3]
            x3, y3 = lines[line_id + 1][coords_second_id], lines[line_id + 1][coords_second_id + 1]
            x4, y4 = lines[line_id + 1][coords_second_id + 2], lines[line_id + 1][coords_second_id + 3]

            cv.fillPoly(mask, [np.array([[x1, y1], [x2, y2], [x4, y4], [x3, y3]])], lane_id + 1, lineType = cv.LINE_AA)

            if coords_first_id + 4 < len(lines[line_id]):
                    coords_first_id += 2
            
            if coords_second_id + 4 < len(lines[line_id + 1]):
                    coords_second_id += 2

            if coords_first_id + 4 == len(lines[line_id]) and coords_second_id + 4 == len(lines[line_id + 1]):
                    x1, y1 = lines[line_id][coords_first_id], lines[line_id][coords_first_id + 1]
                    x2, y2 = lines[line_id][coords_first_id + 2], lines[line_id][coords_first_id + 3]
                    x3, y3 = lines[line_id + 1][coords_second_id], lines[line_id + 1][coords_second_id + 1]
                    x4, y4 = lines[line_id + 1][coords_second_id + 2], lines[line_id + 1][coords_second_id + 3]

                    cv.fillPoly(mask, [np.array([[x1, y1], [x2, y2], [x4, y4], [x3, y3]])], lane_id + 1, lineType = cv.LINE_AA)

                    break

        line_id += 1

        lane_id += 1

    return np.stack([mask] * 3, axis=-1), lane_id

def resize_and_pad_mask(mask, target_size=512, pad_value=255):
    mask = mask[:, :, 0]

    original_height, original_width = mask.shape

    # Обчислюємо масштаб
    scale = min(target_size / original_height, target_size / original_width)
    new_height = int(round(original_height * scale))
    new_width = int(round(original_width * scale))

    # Resize з NEAREST для збереження lane_id
    resized = cv.resize(mask, (new_width, new_height), interpolation=cv.INTER_NEAREST)

    # Обчислюємо паддінг
    pad_top = (target_size - new_height) // 2
    pad_bottom = target_size - new_height - pad_top
    pad_left = (target_size - new_width) // 2
    pad_right = target_size - new_width - pad_left

    # Паддінг
    padded = cv.copyMakeBorder(
        resized,
        pad_top, pad_bottom,
        pad_left, pad_right,
        borderType=cv.BORDER_CONSTANT,
        value=pad_value
    )

    return padded.astype(np.uint8)

mask, lanes_num = data_process_mask(lines, image)

mask_res = resize_and_pad_mask(mask)

colored_mask = mask_res.copy()

print(mask_res.shape)

colored_mask = cv.applyColorMap(mask_res * (255 // (lanes_num + 2)), cv.COLORMAP_JET)

cv.imshow("Mask", colored_mask)

cv.waitKey(0)
cv.destroyAllWindows()

(512, 512)


In [17]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

# Шляхи до файлів
img_path = r"C:\Users\maksym.kashchuk\Desktop\auto-os-ml\datasets\CuLane\train-validation\driver_161_90frame\06031152_0826.MP4\00000.jpg"
lines_path = r"C:\Users\maksym.kashchuk\Desktop\auto-os-ml\datasets\CuLane\train-validation\driver_161_90frame\06031152_0826.MP4\00000.lines.txt"

def data_process_mask(lines_file, image_file):
    mask_wdith, mask_height, _ = cv.imread(image_file).shape

    lines = []

    line_id = 0

    thickness = 3

    mask = np.zeros((mask_wdith, mask_height), dtype=np.uint8)

    with open(lines_file, 'r') as file:
        lines = [list(map(int, map(float, line.split()))) for line in file]

    for line in lines:
        i = 0

        while i < len(line) - 2:
            x1, y1 = line[i], line[i + 1]
            x2, y2 = line[i + 2], line [i + 3]

            cv.line(mask, (int(x1), int(y1)), (int(x2), int(y2)), line_id, thickness)

            i += 2

        line_id += 1

    return mask

def resize_and_pad_mask(mask, target_size=512, pad_value=255):
    mask = mask[:, :]

    original_height, original_width = mask.shape

    scale = min(target_size / original_height, target_size / original_width)
    new_height = int(round(original_height * scale))
    new_width = int(round(original_width * scale))

    resized = cv.resize(mask, (new_width, new_height), interpolation=cv.INTER_NEAREST)

    pad_top = (target_size - new_height) // 2
    pad_bottom = target_size - new_height - pad_top
    pad_left = (target_size - new_width) // 2
    pad_right = target_size - new_width - pad_left

    padded = cv.copyMakeBorder(
        resized,
        pad_top, pad_bottom,
        pad_left, pad_right,
        borderType=cv.BORDER_CONSTANT,
        value=pad_value
    )

    return padded.astype(np.uint8)

# Зчитуємо зображення
img = cv.imread(img_path)
if img is None:
    raise FileNotFoundError(f"Не можу знайти або відкрити зображення за шляхом {img_path}")

# Конвертуємо BGR у RGB для matplotlib
img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)

lanes_img = data_process_mask(lines_path, img_path)

lanes_img = resize_and_pad_mask(lanes_img)

import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
print(lanes_img)

# Візуалізуємо
plt.figure(figsize=(15, 8))
plt.imshow(lanes_img)
plt.title("Зображення з лініями")
plt.axis("off")
plt.show()


NameError: name 'numpy' is not defined